# Projet de Génomique : Mapping 
### Fait par EL HOUMA Mohamed, DEKARI Adil et BIYIK Kenan 

Durant ce projet, l'objectif était de développer une stratégie de << mapping >> de séquences 
génomiques courtes, appelées “reads” (ou lectures), sur un génome de référence donné.

Nous allons donc vous présenter de manière détaillée les différentes étapes d'élaboration de ce projet.

## I - Initialisation des données

Le jeu de données qui nous a été fournis était sous le format FastQ. Notre première mission était donc de récupérer notre génome et de le formater de façon à le rendre utilisable dans la suite de notre projet.
Pour cela, nous nous sommes servi de la bibliothèque Biopython qui présente de nombreuses fonctions qui vont nous faciliter la manipulation de notre jeu de données.

In [ ]:
%pip install biopython

In [ ]:
from Bio import SeqIO
from pathlib import Path
import gzip, zipfile
import time

def genome(fasta_file):
        sequences = []
        open_func = gzip.open if fasta_file.endswith(".gz") else open
        
        with open_func(fasta_file, "rt") as handle:
            for record in SeqIO.parse(handle, "fasta"):
                sequences += str(record.seq)

        T = "".join(sequences).upper()

        return T

Ici, nous importons les bibliothèques nécessaires et définissons une fonction read_genome

Cette fonction lira notre fichier fasta et le formatera en une unique chaîne de caractères que nous pourrons ensuite indexer avec DC3.

Maintenant que nos données sont utilisables, passons à la suite de ce projet.

## II - Création d'une suffix array

Dans cette partie, notre but est de définir une structure de données permettant de 
chercher des mots de longueur fixe à partir d’un texte d'une certaines longueurs.
Ici, on va former ce qu'on appelle une << suffix array >>. 
Pour faire simple, c'est un tableau contenant les positions de tous les suffixes de la chaîne, triés par ordre lexicographique.
Cette structure de données va nous permettre de rechercher de manière assez facile les reads découpés en k-mers sur notre génome de référence.
Pour créer ce suffix array, on va se servir de l'algorithme DC3 dont vous trouverez le programme ci-dessous : 

In [ ]:
# L'algorithme DC3 pour la construction du Suffix Array est en O(n) 


import numpy as np

start_time = time.perf_counter()

def print_runtime():
    elapsed = time.perf_counter() - start_time
    return elapsed

def presence_doublons(l): #pour la vérification de doublons
    return len(l) != len(set(l))

def radix_sort(L, num_keys):
    if type(L[0][0])==str:
        for i in range(len(L)):
            col = []
            for x in L[i]:
                if type(x)==str:
                    L[i] += [ord(x)]
                else:
                    L[i] += [x]

    for key_index in range(num_keys - 1, -1, -1):
        max_val = max(x[key_index] for x in L)
        count = [0] * (max_val + 2)
        for x in L:
            count[x[key_index] + 1] += 1
        for i in range(1, len(count)):
            count[i] += count[i - 1]
        res = [0] * len(L)
        for x in L:
            res[count[x[key_index]]] = x
            count[x[key_index]] += 1

        L = res
    return L

def merge(index_0, index_12, liste_T, iterations, rank_12):
    index_012=[]
    i_a=0
    i_b=0

    while i_a<len(index_0) and i_b<len(index_12):

        a = index_0[i_a]
        b = index_12[i_b]


        if liste_T[iterations-1][a]<liste_T[iterations-1][b]:
            index_012+=[a]
            i_a+=1


        elif liste_T[iterations-1][a]>liste_T[iterations-1][b]:
            index_012+=[b]
            i_b+=1


        else :
            if b%3==1 :
                if rank_12.get(a+1, -1) < rank_12.get(b+1, -1):
                    index_012+=[a]
                    i_a+=1

                else :
                    index_012+=[b]
                    i_b+=1

            else :
                if liste_T[iterations-1][a+1]<liste_T[iterations-1][b+1]:
                    index_012+=[a]
                    i_a+=1


                elif liste_T[iterations-1][a+1]>liste_T[iterations-1][b+1]:
                    index_012+=[b]
                    i_b+=1


                else:
                    if rank_12.get(a+2, -1) < rank_12.get(b+2, -1):
                        index_012+=[a]
                        i_a+=1

                    else :
                        index_012+=[b]
                        i_b+=1

    if i_a==len(index_0):
        index_012+=index_12[i_b:]
    else:
        index_012+=index_0[i_a:]

    index_012.pop(0)

    return(index_012)

def DC3(T):
    ## ETAPE 1 :
    verif=True
    iterations=0
    liste_T=[T]
    liste_P12=[]
    liste_P0=[]
    liste_P1=[]
    
    while verif:
        if type(T[0])==str:
            T+=['0','0','0']
        else :
            T+=[0,0,0]
        
        P0=[i for i in range (0,len(T)-2,3)]
        P1=[i for i in range (1,len(T)-2,3)]
        P2=[i for i in range (2,len(T)-2,3)]
        P12=P1+P2
        liste_P12+=[P12]
        liste_P0+=[P0]
        liste_P1+=[P1]
        
        if type(T[0])==str:
            base_map = {'A': 1, 'C': 2, 'G': 3, 'T': 4, '0': 0}
            T_array = np.array([base_map[b.upper()] for b in T])
        else:
            T_array = np.asarray(T)
        P12_array = np.asarray(P12)
        n = len(P12)

        R12 = np.empty((n, 5), dtype=np.int64)
        R12[:, 0] = T_array[P12_array]
        R12[:, 1] = T_array[P12_array + 1]
        R12[:, 2] = T_array[P12_array + 2]
        R12[:, 3] = P12_array
        R12[:, 4] = np.arange(n)
        R12=R12.tolist()
        
        R12_sorted=radix_sort(R12,3)
        
        index_12=[R12_sorted[i][3] for i in range (len(R12_sorted))]

        ordre=[1]
        c=1
        for i in range (1,len(R12_sorted)):
            if R12_sorted[i][:3]!=R12_sorted[i-1][:3]:
                c+=1
            ordre+=[c]
        
        T_prime=[0]*len(P12)
        for i in range (len(R12_sorted)):
            T_prime[R12_sorted[i][4]]=ordre[i]
        T=T_prime
        liste_T+=[T]
        iterations+=1
        verif = presence_doublons(T)

    while iterations!=1:
        ## ETAPE 2 :
        rank_12 = {val: i for i, val in enumerate(index_12)}

        R0 = []
        for i in range(len(liste_P1[iterations-1])):
            var_1 = liste_T[iterations-1][liste_P0[iterations-1][i]]
            var_2 = rank_12.get(liste_P1[iterations-1][i], -1) + 1
            R0.append([var_1, var_2, liste_P0[iterations-1][i]])

        if len(liste_P1[iterations-1])<len(liste_P0[iterations-1]):
            R0+=[[liste_T[iterations-1][liste_P0[iterations-1][-1]],0,liste_P0[iterations-1][i]]]
        
        R0_sorted=radix_sort(R0,2)
        
        index_0 = [r[2] for r in R0_sorted]
        ordre_0 = [i for i in range (1,len(index_0)+1)]
        
        ## ETAPE 3 :
        rank_12 = {val: pos for pos, val in enumerate(index_12)}

        index_012 = merge(index_0, index_12, liste_T, iterations, rank_12)
        iterations-=1

        P12_n=liste_P12[iterations-1]
        index_12=[P12_n[i] for i in index_012]

    end_time=print_runtime()
    
    return index_12, end_time


Notre suffix array est fin prêt. On peut donc passer à l'étape suivante.

## III - Découpage des lectures

Pour faire le mapping de nos lectures de manière précise, on va les découper en k-mers dont on va définir la taille (celle-ci va être choisis selon différents facteurs, essentiellement le temps de recherche, la mémoire utilisée et l'ambiguité du mapping).

In [ ]:
def k_mers(seq, k):
    return [seq[i:i+k] for i in range(len(seq) - k + 1)]

read = "AGCTTAGC"
k = 3
print(k_mers(read, k))

## IV - Mapping

Notre structure étant créer, on peut donc passer au mapping, c'est-à-dire à la recherche des k-mers dans le suffix array qu'on a créé précedemment.
Pour faire cela, on va utiliser ce qu'on appelle la Burrows-Wheeler Transform (BWT).

Dans un premier temps, on fabrique tous nos circular shifts :

In [ ]:
def circular_shift(text,k):
    return text[-k:]+text[:-k]

Ensuite, on applique la BWT : 

In [ ]:
def BWT(text, suffix_array):
    bwt = [''] * len(suffix_array)
    for i in range(len(suffix_array)):
        pos = suffix_array[i]
        if pos == 0:
            bwt[i] = text[-1] 
        else:
            bwt[i] = text[pos - 1]
    return "".join(bwt)

Inversion de la séquence pour rechercher dans la séquence complémentaire: 

In [ ]:
def complémentaire(seq):
    """Donne le brin complémentaire inversé."""
    a_inverser = str.maketrans("ACGTN$", "TGCAN$", " ") 
    return seq.translate(a_inverser)[::-1]


Recherche avec l'algorithme de backward search (c correspond à la position, Occ au nombre): 

In [ ]:
from collections import defaultdict

# Définit la fonction pour créer la table C 
def construire_C(bwt):
    counts = defaultdict(int)
    # Parcourt chaque caractère dans la BWT 
    for c in bwt:
        counts[c] += 1
    # Récupère tous les caractères uniques et les trie par ordre lexicographique
    chars = sorted(counts.keys())
    # Initialise le dictionnaire C final
    C = {}
    total = 0
    for c in chars:
        C[c] = total
        # Ajoute le nombre d'occurrences de c au total pour la prochaine itération
        total += counts[c]
    # Renvoie la table C complétée
    return C

# Définit la fonction pour créer la table Occ (occurrences)
def construire_Occ(bwt):
    # Récupère l'alphabet de la BWT
    chars = sorted(set(bwt))
    occ = {c: [0] for c in chars}
    for ch in bwt:
        for c in chars:
            occ[c].append(occ[c][-1] + (1 if c == ch else 0))
    return occ

# Définit la fonction d'accès rapide à la table Occ (combien de c avant la position)
def Occ(occ, c, pos):
    if pos < 0:
        return 0
    # Renvoie le compte précalculé à la position pos 
    return occ[c][pos+1]

# Définit la fonction de recherche arrière (Backward Search)
def backward_search(pattern, bwt, suffix_array, C, Occ_table):  
    # Initialise le pointeur de début de plage (indice Suffix Array)
    start = 0
    # Initialise le pointeur de fin de plage (indice Suffix Array)
    end = len(bwt) - 1

    # Parcourt le pattern (le motif) à l'envers, du dernier caractère au premier
    for i in range(len(pattern)-1, -1, -1):
        # Récupère le caractère actuel du motif
        c = pattern[i]
        # Si ce caractère n'est pas dans le génome
        if c not in C:
            # le motif n'existe pas, on renvoie 0 occurrence et une liste vide
            return 0, []  
        
        # Calcule la nouvelle position de début
        pos_start = C[c] + Occ(Occ_table, c, start - 1)
        # Calcule la nouvelle position de fin
        pos_end = C[c] + Occ(Occ_table, c, end) - 1
        
        start = pos_start
        end = pos_end

        if start > end:
            #le motif n'existe pas on arrête
            return 0, []

    # Calcule le nombre total d'occurrences
    count = end - start + 1
    # Récupère les positions de départ réelles dans le génome en consultant le Suffix Array
    positions = [suffix_array[i] for i in range(start, end + 1)]
    # Renvoie le nombre d'occurrences et la liste des positions
    return count, positions

# Définit la fonction pour extraire un k-mer à une position i
def get_kmer_at_position(genome_text, i, k):
    
    start_index = i - 1
    
    # Calcule l'index de fin
    end_index = start_index + k
    
    # Extrait et renvoie la sous-chaîne (le k-mer)
    return genome_text[start_index : end_index]

Pour le brin complémentaire : soit on applique DC3 2x pour le brin sens et le brin antisens, soit on colle le brin sens et le brin antisens pour avoir un grand génome et appliquait la recherche dessus.
Pas besoin de prendre le complémentaire de la BWT

Voir BWT non naïve pour le deuxième tiré de la mise en oeuvre

In [ ]:
# 0. Paramètres
# On définit ici la taille du k-mer
taille_kmer = 25

# Importation des fichiers (A changer pour l'utilisateur)
fasta_file = r"C:\\Users\\melho\\Downloads\\GCF_000002765.5_GCA_000002765_genomic (1).fna.gz"
fastq_file = r"C:\\Users\\melho\\Downloads\\single_Pfal_dat (1).fq.gz"


# 1. Charger le génome ---
T = genome(fasta_file) 

# 2. Construire le Suffix Array (DC3)
suffix_array, runtime = DC3(list(T))
print(f"DC3 a pris {runtime} secondes.")
print(f"la suffix array est :")
print(suffix_array[:100])
print('')

# 3. Construire la BWT 
bwt = BWT(T, suffix_array) 
C_table = construire_C(bwt)
Occ_table = construire_Occ(bwt)
    
reads_trouves = 0
total_reads = 0
    
# On prend chaque read de notre fichier fastq_file et le recherche dans notre génome de référence

# 4. Boucle de Mapping ---
with gzip.open(fastq_file, "rt") as handle:
    for record in SeqIO.parse(handle, "fastq"):
        total_reads += 1
        read_id = record.id
        read_seq = str(record.seq)
            
        # On ne prend que le premier k-mer comme "seed"
        seed = read_seq[:taille_kmer]
            
        # 6. Recherche du k-mer sur le brin sens (+)
        count_fwd, positions_fwd = backward_search(seed, bwt, suffix_array, C_table, Occ_table)
            
        # 7. Recherche du k-mer sur le brin complémentaire (-)
        seed_rev = complémentaire(seed)
        count_rev, positions_rev = backward_search(seed_rev, bwt, suffix_array, C_table, Occ_table)
            
#       8. Affichage des résultats
        if count_fwd > 0:
            reads_trouves += 1
            print(f"Read '{read_id}' TROUVÉ (Brin '+') : {count_fwd} aux positions {positions_fwd}")
            
        if count_rev > 0:
            reads_trouves += 1
            print(f"Read '{read_id}' TROUVÉ (Brin '-') : {count_rev} aux positions {positions_rev}")

    print(f"Mapping terminé")
    print(f"Reads analysés : {total_reads}")
    print(f"Reads mappés : {reads_trouves}")

In [ ]:
# Un extrait de la séquence test :
print(T[:100])

In [ ]:
# Si l'on cherche un motif spécifique dans le génome     
# Spécifiez la séquence à rechercher
mon_pattern = list(input("séquences à rechercher : "))
print(mon_pattern)
count, positions = backward_search(mon_pattern, bwt, suffix_array, C_table, Occ_table)

if count > 0:
    print(f"Motif trouvé {count} fois et aux positions {positions}")
else:
    print("Motif non trouvé.")


In [ ]:
#  Si l'on cherche à extraire un k-mer à une position donnée
#  Spécifiez la position et la longueur
 
position_i = int(input("Entrez la position i : ")) # i >= 1
longueur_k = int(input("Entrez la longueur k : ")) # k > 0 

séquence_rechercher = get_kmer_at_position(T, position_i, longueur_k)

print(f"La séquence recherchée est : {séquence_rechercher}")